In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
from nltk.stem import PorterStemmer

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

In [4]:
data = pd.read_excel('training_set_rel3.xls')
df = data[data['essay_set']==1]
len(df)

1783

# Spliting data set for train and test 

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(df['essay'],df['domain1_score'],test_size=0.4,random_state=0)

In [9]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((1069,), (1069,), (714,), (714,))

# Streaming function create

In [10]:
ps = PorterStemmer()
def Prepos(w):
    return ps.stem(w)

# Tfidf table create

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
dtm = TfidfVectorizer(min_df=1,stop_words='english',preprocessor=Prepos,ngram_range=(1,2)).fit(X_train)

In [92]:
len(dtm.get_feature_names())

107194

In [93]:
xt_data = dtm.transform(X_train)
xtest_data = dtm.transform(X_test)
xtest_data

<714x107194 sparse matrix of type '<class 'numpy.float64'>'
	with 123474 stored elements in Compressed Sparse Row format>

In [94]:
pd.DataFrame(xt_data.toarray(),columns=dtm.get_feature_names()).head()

,000,000 dollar,00pm,00pm computer,aa,aa day,aamous,aamous social,aare,aare really,...,zone,zone computer,zoo,zoo didn,zoo learn,zoo location2,zoo preventing,zoo right,zoos,zoos near
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# LSA create 

In [95]:
from sklearn.decomposition import TruncatedSVD

In [96]:
lsa = TruncatedSVD(110)

In [97]:
dt_lsa = lsa.fit_transform(xt_data)
dtest_lsa = lsa.fit_transform(xtest_data)

# For normalizer

In [98]:
from sklearn.preprocessing import Normalizer

In [99]:
dt_lsa = Normalizer(copy=False).fit_transform(dt_lsa)
dtest_lsa = Normalizer(copy=False).fit_transform(dtest_lsa)

In [100]:
dt_lsa

array([[ 0.49058687,  0.38833279,  0.30593317, ..., -0.04757972,
        -0.01774354,  0.10576512],
       [ 0.4757297 ,  0.07940524, -0.17163541, ...,  0.01347813,
         0.00699361, -0.08422302],
       [ 0.35741305, -0.02142743, -0.09872914, ..., -0.16731367,
         0.0376829 , -0.05744661],
       ..., 
       [ 0.48707651, -0.07520135,  0.00761801, ..., -0.06547797,
        -0.04801355,  0.03516379],
       [ 0.36789928, -0.03076698, -0.11149051, ..., -0.13489777,
         0.07277875,  0.06254678],
       [ 0.35956315,  0.01339486, -0.05985203, ...,  0.0028235 ,
        -0.08114176,  0.04577818]])

# Create sparce matrix from array

In [101]:
from scipy import sparse

In [102]:
train_data = sparse.csr_matrix(dt_lsa)
test_data = sparse.csr_matrix(dtest_lsa)

In [103]:
train_data
test_data

<714x110 sparse matrix of type '<class 'numpy.float64'>'
	with 78540 stored elements in Compressed Sparse Row format>

# Model create

In [104]:
from sklearn import svm

In [105]:
model = svm.SVC(kernel='linear',C=1,random_state=0)

In [106]:
model.fit(train_data,Y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [107]:
predictions=model.predict(test_data)

In [108]:
from sklearn.metrics import accuracy_score

In [109]:
accuracy_score(Y_test,predictions)

0.39775910364145656

In [110]:
model.score(test_data,Y_test)

0.39775910364145656